In [35]:
# -*- coding: utf-8 -*-

In [1]:
tutorial='https://notebook.community/jmschrei/pomegranate/tutorials/B_Model_Tutorial_4_Bayesian_Networks'
#!pip install pomegranate

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2


In [36]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

NameError: name 'reload' is not defined

In [34]:
import chardet
print(chardet.detect(df.관할해경서[0].encode()))

{'encoding': 'utf-8', 'confidence': 0.7525, 'language': ''}


In [2]:
import pandas as pd

In [33]:
df=pd.read_csv('./Data/해양경찰청_해상조난사고_상세데이터_현황_20191231.csv', encoding='euckr')
df.rename(columns={'선 종':'선종'},inplace=True)
df = df[['발생일시','관할해경서','발생해역','기상상태','발생원인','발생유형','선종']]

In [4]:
df

,발생일시,관할해경서,발생해역,기상상태,발생원인,발생유형,선종
0,2019-12-31 22:04,포항,영해,양호,관리소홀,기타,모터보트
1,2019-12-31 15:13,포항,영해,양호,운항부주의,충돌,어선
2,2019-12-31 15:13,포항,영해,양호,운항부주의,충돌,화물선
3,2019-12-31 9:33,통영,협수로,양호,운항부주의,부유물감김,어선
4,2019-12-31 8:38,군산,항계 내,양호,관리소홀,침수,어선
...,...,...,...,...,...,...,...
3815,2019-01-01 14:08,인천,영해,양호,운항부주의,부유물감김,모터보트
3816,2019-01-01 9:14,인천,협수로,양호,운항부주의,침수,어선
3817,2019-01-01 7:59,완도,항계 내,양호,운항부주의,좌초/좌주,모터보트
3818,2019-01-01 3:31,태안,영해,양호,운항부주의,침몰,기타


In [5]:
df['관할해경서'].value_counts()

목포     462
통영     375
여수     307
제주     244
서귀포    231
인천     223
부산     220
포항     207
평택     199
창원     195
군산     169
태안     167
울산     162
동해     153
완도     151
보령     138
속초      77
울진      73
부안      67
Name: 관할해경서, dtype: int64

In [6]:
df['발생해역'].value_counts()

영해             2064
항계 내           1201
영해-EEZ          252
협수로             240
공해               49
EEZ-30마일 이내      14
Name: 발생해역, dtype: int64

In [7]:
df['기상상태'].value_counts()

양호       3385
저시정       115
풍랑주의보      96
태풍경보       86
황천5급       59
황천6급       46
황천4급       17
풍랑경보        9
태풍주의보       7
Name: 기상상태, dtype: int64

In [8]:
df['발생원인'].value_counts()

정비불량            1483
운항부주의           1297
관리소홀             439
기상악화             173
안전부주의            158
연료고갈             113
기타                95
화기취급부주의           52
적재불량               7
재질불량               2
기상악화(인명은 기타)       1
Name: 발생원인, dtype: int64

In [9]:
df['발생유형'].value_counts()

기관손상         1011
부유물감김         506
충돌            383
운항저해          306
추진기손상         305
침수            238
좌초/좌주         206
작업 중 인명사상     148
화재            128
기타            111
키손상           108
전복            104
해양오염           88
접촉             71
침몰             54
방향상실           39
속구손상            6
폭발              4
시설물손상           3
침수(인명은기타)       1
Name: 발생유형, dtype: int64

In [10]:
df['선종'].value_counts()

어선        1985
모터보트       606
낚시어선       306
예부선        205
고무보트       197
기타         131
화물선        116
요트          89
유조선         83
여객선         47
관공선         27
유도선         26
기타(통선)       2
Name: 선종, dtype: int64

### 확률 계산

#### 기상상태

In [11]:
df['기상상태'].value_counts()['양호']

3385

In [37]:
weather_total = len(df['기상상태'])
weather_vc = df['기상상태'].value_counts()
weather_p = {}

for i in df['기상상태'].unique():
    weather_p[i] = round(weather_vc[i]/weather_total,4)
    
weather_p

{'양호': 0.8861,
 '풍랑주의보': 0.0251,
 '황천5급': 0.0154,
 '황천6급': 0.012,
 '황천4급': 0.0045,
 '풍랑경보': 0.0024,
 '저시정': 0.0301,
 '태풍경보': 0.0225,
 '태풍주의보': 0.0018}

#### 발생일시|기상상태

#### 발생해역|기상상태

In [38]:
ocean_total = len(df['발생해역'])
ocean_vc = df['발생해역'].value_counts()
ocean_p = []

for i in df['기상상태'].unique():
    for j in df['발생해역'].unique():
        cnt=len(df[(df.기상상태==i) & (df.발생해역==j)])
        ll = []
        ll.append(i)
        ll.append(j)
        ll.append(round(cnt/ocean_total,4))
        ocean_p.append(ll)
        
ocean_p

[['양호', '영해', 0.4916],
 ['양호', '협수로', 0.062],
 ['양호', '항계 내', 0.2694],
 ['양호', '영해-EEZ', 0.0492],
 ['양호', '공해', 0.0105],
 ['양호', 'EEZ-30마일 이내', 0.0034],
 ['풍랑주의보', '영해', 0.0079],
 ['풍랑주의보', '협수로', 0.0005],
 ['풍랑주의보', '항계 내', 0.0099],
 ['풍랑주의보', '영해-EEZ', 0.0045],
 ['풍랑주의보', '공해', 0.0021],
 ['풍랑주의보', 'EEZ-30마일 이내', 0.0003],
 ['황천5급', '영해', 0.005],
 ['황천5급', '협수로', 0.0],
 ['황천5급', '항계 내', 0.0047],
 ['황천5급', '영해-EEZ', 0.0058],
 ['황천5급', '공해', 0.0],
 ['황천5급', 'EEZ-30마일 이내', 0.0],
 ['황천6급', '영해', 0.0099],
 ['황천6급', '협수로', 0.0],
 ['황천6급', '항계 내', 0.001],
 ['황천6급', '영해-EEZ', 0.001],
 ['황천6급', '공해', 0.0],
 ['황천6급', 'EEZ-30마일 이내', 0.0],
 ['황천4급', '영해', 0.001],
 ['황천4급', '협수로', 0.0],
 ['황천4급', '항계 내', 0.0008],
 ['황천4급', '영해-EEZ', 0.0024],
 ['황천4급', '공해', 0.0003],
 ['황천4급', 'EEZ-30마일 이내', 0.0],
 ['풍랑경보', '영해', 0.0005],
 ['풍랑경보', '협수로', 0.0],
 ['풍랑경보', '항계 내', 0.0016],
 ['풍랑경보', '영해-EEZ', 0.0003],
 ['풍랑경보', '공해', 0.0],
 ['풍랑경보', 'EEZ-30마일 이내', 0.0],
 ['저시정', '영해', 0.0202],
 ['저시정', '협수로', 0.0],
 [

In [152]:
0.8861*0.4916

0.43560676

#### 관할해경서|발생해역

In [39]:
# juris_total = len(df['관할해경서'])
# juris_vc = df['관할해경서'].value_counts()
# juris_p = []

# for i in df['기상상태'].unique():
#     for j in df['발생해역'].unique():
#         for k in df['관할해경서'].unique():
#             cnt=len(df[(df.기상상태==i) & (df.발생해역==j) & (df.관할해경서==k)])
#             ll = []
#             ll.append(i)
#             ll.append(j)
#             ll.append(k)
#             ll.append(round(cnt/juris_total,4))
#             juris_p.append(ll)
        
# juris_p

In [40]:
juris_total = len(df['관할해경서'])
juris_vc = df['관할해경서'].value_counts()
juris_p = []


for j in df['발생해역'].unique():
    for k in df['관할해경서'].unique():
        cnt=len(df[(df.발생해역==j) & (df.관할해경서==k)])
        ll = []
        ll.append(j)
        ll.append(k)
        ll.append(round(cnt/juris_total,4))
        juris_p.append(ll)
        
juris_p

[['영해', '포항', 0.0359],
 ['영해', '통영', 0.0243],
 ['영해', '군산', 0.0249],
 ['영해', '여수', 0.0236],
 ['영해', '서귀포', 0.0296],
 ['영해', '인천', 0.0387],
 ['영해', '보령', 0.0264],
 ['영해', '동해', 0.0212],
 ['영해', '부산', 0.0249],
 ['영해', '울진', 0.011],
 ['영해', '목포', 0.0848],
 ['영해', '속초', 0.0118],
 ['영해', '완도', 0.0029],
 ['영해', '제주', 0.0393],
 ['영해', '평택', 0.0408],
 ['영해', '창원', 0.0314],
 ['영해', '울산', 0.0188],
 ['영해', '태안', 0.0353],
 ['영해', '부안', 0.0147],
 ['협수로', '포항', 0.0],
 ['협수로', '통영', 0.0495],
 ['협수로', '군산', 0.0],
 ['협수로', '여수', 0.0013],
 ['협수로', '서귀포', 0.0],
 ['협수로', '인천', 0.0008],
 ['협수로', '보령', 0.0018],
 ['협수로', '동해', 0.0],
 ['협수로', '부산', 0.001],
 ['협수로', '울진', 0.0],
 ['협수로', '목포', 0.0021],
 ['협수로', '속초', 0.0003],
 ['협수로', '완도', 0.0003],
 ['협수로', '제주', 0.0],
 ['협수로', '평택', 0.0005],
 ['협수로', '창원', 0.0052],
 ['협수로', '울산', 0.0],
 ['협수로', '태안', 0.0],
 ['협수로', '부안', 0.0],
 ['항계 내', '포항', 0.0073],
 ['항계 내', '통영', 0.0194],
 ['항계 내', '군산', 0.016],
 ['항계 내', '여수', 0.0555],
 ['항계 내', '서귀포', 0.0097],
 ['항계 내',

##### 발생원인|발생해역

In [41]:
cause_total = len(df['발생원인'])
cause_vc = df['발생원인'].value_counts()
cause_p = []


for j in df['발생해역'].unique():
    for k in df['발생원인'].unique():
        cnt=len(df[(df.발생해역==j) & (df.발생원인==k)])
        ll = []
        ll.append(j)
        ll.append(k)
        ll.append(round(cnt/cause_total,4))
        cause_p.append(ll)
        
cause_p

[['영해', '관리소홀', 0.0236],
 ['영해', '운항부주의', 0.1995],
 ['영해', '정비불량', 0.2487],
 ['영해', '안전부주의', 0.0209],
 ['영해', '화기취급부주의', 0.0034],
 ['영해', '연료고갈', 0.0188],
 ['영해', '기타 ', 0.0105],
 ['영해', '기상악화', 0.0131],
 ['영해', '기상악화(인명은 기타)', 0.0],
 ['영해', '적재불량', 0.0016],
 ['영해', '재질불량', 0.0003],
 ['협수로', '관리소홀', 0.0034],
 ['협수로', '운항부주의', 0.0233],
 ['협수로', '정비불량', 0.0301],
 ['협수로', '안전부주의', 0.0008],
 ['협수로', '화기취급부주의', 0.0008],
 ['협수로', '연료고갈', 0.0031],
 ['협수로', '기타 ', 0.0008],
 ['협수로', '기상악화', 0.0005],
 ['협수로', '기상악화(인명은 기타)', 0.0],
 ['협수로', '적재불량', 0.0],
 ['협수로', '재질불량', 0.0],
 ['항계 내', '관리소홀', 0.088],
 ['항계 내', '운항부주의', 0.0817],
 ['항계 내', '정비불량', 0.0754],
 ['항계 내', '안전부주의', 0.0105],
 ['항계 내', '화기취급부주의', 0.0079],
 ['항계 내', '연료고갈', 0.0068],
 ['항계 내', '기타 ', 0.0126],
 ['항계 내', '기상악화', 0.0309],
 ['항계 내', '기상악화(인명은 기타)', 0.0003],
 ['항계 내', '적재불량', 0.0003],
 ['항계 내', '재질불량', 0.0003],
 ['영해-EEZ', '관리소홀', 0.0],
 ['영해-EEZ', '운항부주의', 0.0275],
 ['영해-EEZ', '정비불량', 0.027],
 ['영해-EEZ', '안전부주의', 0.0073],
 ['영해

#### 선종|발생해역

In [42]:
ship_total = len(df['선종'])
ship_vc = df['선종'].value_counts()
ship_p = []


for j in df['발생해역'].unique():
    for k in df['선종'].unique():
        cnt=len(df[(df.발생해역==j) & (df.선종==k)])
        ll = []
        ll.append(j)
        ll.append(k)
        ll.append(round(cnt/ship_total,4))
        ship_p.append(ll)
        
ship_p

[['영해', '모터보트', 0.0982],
 ['영해', '어선', 0.2649],
 ['영해', '화물선', 0.0134],
 ['영해', '기타', 0.0123],
 ['영해', '낚시어선', 0.055],
 ['영해', '유조선', 0.0047],
 ['영해', '예부선', 0.0217],
 ['영해', '여객선', 0.005],
 ['영해', '관공선', 0.0031],
 ['영해', '요트', 0.0162],
 ['영해', '고무보트', 0.0419],
 ['영해', '유도선', 0.0039],
 ['영해', '기타(통선)', 0.0],
 ['협수로', '모터보트', 0.0134],
 ['협수로', '어선', 0.0298],
 ['협수로', '화물선', 0.0005],
 ['협수로', '기타', 0.001],
 ['협수로', '낚시어선', 0.0073],
 ['협수로', '유조선', 0.0008],
 ['협수로', '예부선', 0.0037],
 ['협수로', '여객선', 0.001],
 ['협수로', '관공선', 0.0016],
 ['협수로', '요트', 0.0005],
 ['협수로', '고무보트', 0.0024],
 ['협수로', '유도선', 0.0008],
 ['협수로', '기타(통선)', 0.0],
 ['항계 내', '모터보트', 0.0471],
 ['항계 내', '어선', 0.1503],
 ['항계 내', '화물선', 0.0134],
 ['항계 내', '기타', 0.0196],
 ['항계 내', '낚시어선', 0.0165],
 ['항계 내', '유조선', 0.0152],
 ['항계 내', '예부선', 0.028],
 ['항계 내', '여객선', 0.006],
 ['항계 내', '관공선', 0.0024],
 ['항계 내', '요트', 0.006],
 ['항계 내', '고무보트', 0.0073],
 ['항계 내', '유도선', 0.0021],
 ['항계 내', '기타(통선)', 0.0005],
 ['영해-EEZ', '모터보트', 0.0],
 ['

#### 발생유형|발생원인

In [43]:
type_total = len(df['발생유형'])
type_vc = df['발생유형'].value_counts()
type_p = []


for j in df['발생원인'].unique():
    for k in df['발생유형'].unique():
        cnt=len(df[(df.발생해역==j) & (df.선종==k)])
        ll = []
        ll.append(j)
        ll.append(k)
        ll.append(round(cnt/type_total,4))
        type_p.append(ll)
        
type_p

[['관리소홀', '기타', 0.0],
 ['관리소홀', '충돌', 0.0],
 ['관리소홀', '부유물감김', 0.0],
 ['관리소홀', '침수', 0.0],
 ['관리소홀', '침몰', 0.0],
 ['관리소홀', '해양오염', 0.0],
 ['관리소홀', '화재', 0.0],
 ['관리소홀', '작업 중 인명사상', 0.0],
 ['관리소홀', '기관손상', 0.0],
 ['관리소홀', '추진기손상', 0.0],
 ['관리소홀', '좌초/좌주', 0.0],
 ['관리소홀', '키손상', 0.0],
 ['관리소홀', '운항저해', 0.0],
 ['관리소홀', '접촉', 0.0],
 ['관리소홀', '전복', 0.0],
 ['관리소홀', '침수(인명은기타)', 0.0],
 ['관리소홀', '방향상실', 0.0],
 ['관리소홀', '폭발', 0.0],
 ['관리소홀', '시설물손상', 0.0],
 ['관리소홀', '속구손상', 0.0],
 ['운항부주의', '기타', 0.0],
 ['운항부주의', '충돌', 0.0],
 ['운항부주의', '부유물감김', 0.0],
 ['운항부주의', '침수', 0.0],
 ['운항부주의', '침몰', 0.0],
 ['운항부주의', '해양오염', 0.0],
 ['운항부주의', '화재', 0.0],
 ['운항부주의', '작업 중 인명사상', 0.0],
 ['운항부주의', '기관손상', 0.0],
 ['운항부주의', '추진기손상', 0.0],
 ['운항부주의', '좌초/좌주', 0.0],
 ['운항부주의', '키손상', 0.0],
 ['운항부주의', '운항저해', 0.0],
 ['운항부주의', '접촉', 0.0],
 ['운항부주의', '전복', 0.0],
 ['운항부주의', '침수(인명은기타)', 0.0],
 ['운항부주의', '방향상실', 0.0],
 ['운항부주의', '폭발', 0.0],
 ['운항부주의', '시설물손상', 0.0],
 ['운항부주의', '속구손상', 0.0],
 ['정비불량', '기타', 0.0],
 ['정

### 베이지안네트워크

In [44]:
from pomegranate import *

In [45]:
weather = Node(DiscreteDistribution(weather_p)
               , name='weather'
              )

ocean = Node(
    ConditionalProbabilityTable(
        ocean_p, 
        [weather.distribution]
    )
    , name='ocean'
)

jurisdiction = Node(
    ConditionalProbabilityTable(
        juris_p,
        [ocean.distribution]
    )
    , name='jurisdiction'
)

cause = Node(
    ConditionalProbabilityTable(
        cause_p,
        [ocean.distribution]
    )
    , name='cause'
)

ship = Node(
    ConditionalProbabilityTable(
        ship_p,
        [ocean.distribution]
    )
    , name='ship'
)

accident_type = Node(
    ConditionalProbabilityTable(
        type_p,
        [cause.distribution]
    )
    , name='accident_type'
)

In [46]:
model = BayesianNetwork()
model.add_states(weather, ocean, jurisdiction, cause, ship, accident_type)

model.add_edge(weather, ocean)
model.add_edge(ocean, jurisdiction)
model.add_edge(ocean, cause)
model.add_edge(ocean, ship)
model.add_edge(cause, accident_type)

model.bake()

### 예측

In [128]:
probability = model.probability([['양호','영해','목포','정비불량','어선',None]])
print(probability)

0.002433594451833785


In [130]:
# order = weather, ocean, jurisdiction, cause, ship, accident_type
probability = model.probability([['양호','영해','목포', None, '어선','기관손상']])
probability

0.0097852611653952

In [137]:
p1=0
for i in df['발생원인'].unique():
    p = model.probability([['양호','영해','목포', i, '어선','기관손상']])
    print(p)
    p1+=p
p1

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0

In [138]:
predictions = model.predict_proba({
    'weather':'양호',
    'ocean' : '영해', 
    'jurisdiction' : '목포', 
    'ship' : '어선', 
    'accident_type' : '기관손상'
    #'cause':'정비불량'
})

# Print predictions for each node
for node, prediction in zip(model.states, predictions):
    if isinstance(prediction, str):
        print('**********************')
        print(f"{node.name}: {prediction}")
        print()
        
    else:
        print('-----------------------')
        print(f"{node.name}")
        for value, probability in prediction.parameters[0].items():
            print(f"    {value}: {probability:.4f}")
        print()

**********************
weather: 양호

**********************
ocean: 영해

**********************
jurisdiction: 목포

-----------------------
cause
    관리소홀: 0.0437
    화기취급부주의: 0.0063
    재질불량: 0.0006
    운항부주의: 0.3692
    안전부주의: 0.0387
    정비불량: 0.4602
    기상악화: 0.0242
    기타 : 0.0194
    연료고갈: 0.0348
    적재불량: 0.0030
    기상악화(인명은 기타): 0.0000

**********************
ship: 어선

**********************
accident_type: 기관손상



In [140]:
df

,발생일시,관할해경서,발생해역,기상상태,발생원인,발생유형,선종
0,2019-12-31 22:04,포항,영해,양호,관리소홀,기타,모터보트
1,2019-12-31 15:13,포항,영해,양호,운항부주의,충돌,어선
2,2019-12-31 15:13,포항,영해,양호,운항부주의,충돌,화물선
3,2019-12-31 9:33,통영,협수로,양호,운항부주의,부유물감김,어선
4,2019-12-31 8:38,군산,항계 내,양호,관리소홀,침수,어선
...,...,...,...,...,...,...,...
3815,2019-01-01 14:08,인천,영해,양호,운항부주의,부유물감김,모터보트
3816,2019-01-01 9:14,인천,협수로,양호,운항부주의,침수,어선
3817,2019-01-01 7:59,완도,항계 내,양호,운항부주의,좌초/좌주,모터보트
3818,2019-01-01 3:31,태안,영해,양호,운항부주의,침몰,기타


In [141]:
predictions = model.predict_proba({
    'weather':'양호',
    'ocean' : '영해', 
    'jurisdiction' : '인천', 
    'ship' : '모터보트', 
    'accident_type' : '부유물감김'
    #'cause':'정비불량'
})

# Print predictions for each node
for node, prediction in zip(model.states, predictions):
    if isinstance(prediction, str):
        print('**********************')
        print(f"{node.name}: {prediction}")
        print()
        
    else:
        print('-----------------------')
        print(f"{node.name}")
        for value, probability in prediction.parameters[0].items():
            print(f"    {value}: {probability:.4f}")
        print()

**********************
weather: 양호

**********************
ocean: 영해

**********************
jurisdiction: 인천

-----------------------
cause
    관리소홀: 0.0437
    화기취급부주의: 0.0063
    재질불량: 0.0006
    운항부주의: 0.3692
    안전부주의: 0.0387
    정비불량: 0.4602
    기상악화: 0.0242
    기타 : 0.0194
    연료고갈: 0.0348
    적재불량: 0.0030
    기상악화(인명은 기타): 0.0000

**********************
ship: 모터보트

**********************
accident_type: 부유물감김



In [151]:
predictions = model.predict_proba({
    'weather':'양호',
    'ocean' : '항계 내', 
    'jurisdiction' : '태안', 
    'ship' : '어선', 
    'accident_type' : '침수'
    #'cause':'정비불량'
})

# Print predictions for each node
for node, prediction in zip(model.states, predictions):
    if isinstance(prediction, str):
        print('**********************')
        print(f"{node.name}: {prediction}")
        print()
        
    else:
        print('-----------------------')
        print(f"{node.name}")
        for value, probability in prediction.parameters[0].items():
            print(f"    {value}: {probability:.4f}")
        print()


**********************
weather: 양호

**********************
ocean: 항계 내

**********************
jurisdiction: 태안

-----------------------
cause
    관리소홀: 0.2796
    화기취급부주의: 0.0251
    재질불량: 0.0010
    운항부주의: 0.2596
    안전부주의: 0.0334
    정비불량: 0.2396
    기상악화: 0.0982
    기타 : 0.0400
    연료고갈: 0.0216
    적재불량: 0.0010
    기상악화(인명은 기타): 0.0010

**********************
ship: 어선

**********************
accident_type: 침수



In [55]:
string="\uc591\ud638"
string.encode('utf-8').decode()

'양호'